# P1 Training Pipeline

This notebook collects the training recipes for the three detectors developed in `p1`: a YOLOv11 model, an RF-DETR model, and an RT-DETR model. 

Each section focuses on data preparation and training only, omitting validation and metric computation.

All training logs are presented in the end of this notebook.

## YOLOv11 Training

In [ ]:
from pathlib import Path
from glob import glob
import json
import os
import shutil
import yaml
from ultralytics import YOLO


def setup_yolo_data(base_data_dir='../../data', output_dir='data_yolo'):
    '''Copy raw assets into a YOLO-compatible folder structure.'''
    base_dir = Path(base_data_dir)
    out_dir = Path(output_dir)

    dirs = {
        'train_labels': out_dir / 'labels' / 'train',
        'train_images': out_dir / 'images' / 'train',
        'val_labels': out_dir / 'labels' / 'val',
        'val_images': out_dir / 'images' / 'val',
    }

    for dir_path in dirs.values():
        dir_path.mkdir(parents=True, exist_ok=True)

    sources = {
        'train_labels': (base_dir / 'TL_KS_BBOX', '*.json'),
        'train_images': (base_dir / 'TS_KS', '*.jpg'),
        'val_labels': (base_dir / 'VL_KS_BBOX', '*.json'),
        'val_images': (base_dir / 'VS_KS', '*.jpg'),
    }

    for key, (src_dir, pattern) in sources.items():
        if src_dir.exists():
            for file in src_dir.glob(pattern):
                shutil.copy2(file, dirs[key] / file.name)

    return {**{k: str(v) for k, v in dirs.items()}, 'output_dir': str(out_dir)}


def convert_json_to_yolo(label_dir: str):
    '''Convert VIA JSON annotations to YOLO txt format in-place.'''
    for file in glob('*.json', root_dir=label_dir):
        with open(os.path.join(label_dir, file), 'r', encoding='utf-8-sig') as f:
            data = json.load(f)
        item = list(data.values())[0]

        img_w = int(item['file_attributes']['img_width'])
        img_h = int(item['file_attributes']['img_height'])

        yolo_labels = []
        for obj in item['regions']:
            x = obj['shape_attributes']['x']
            y = obj['shape_attributes']['y']
            w = obj['shape_attributes']['width']
            h = obj['shape_attributes']['height']
            cx = (x + w / 2) / img_w
            cy = (y + h / 2) / img_h
            nw = w / img_w
            nh = h / img_h
            yolo_labels.append(f'0 {cx} {cy} {nw} {nh}')

        out_file = file.replace('.json', '.txt')
        with open(os.path.join(label_dir, out_file), 'w') as f:
            f.write('\n'.join(yolo_labels))


def create_yolo_dataset_yaml(dataset_root: str, yaml_path: str):
    '''Write the dataset description expected by Ultralytics YOLO.'''
    data_yaml = {
        'path': dataset_root,
        'train': 'images/train',
        'val': 'images/val',
        'nc': 1,
        'names': ['chimney'],
    }

    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml, f, default_flow_style=False)


def train_yolo_model(
    base_data_dir='../../data',
    output_dir='data_yolo',
    epochs=100,
    imgsz=512,
    batch=48,
    device=0,
    model_weights='yolo11x.pt',
):
    '''Full YOLO training pipeline.'''
    paths = setup_yolo_data(base_data_dir, output_dir)

    convert_json_to_yolo(paths['train_labels'])
    convert_json_to_yolo(paths['val_labels'])

    data_yaml_path = Path(paths['output_dir']) / 'dataset.yaml'
    create_yolo_dataset_yaml(paths['output_dir'], data_yaml_path)

    model = YOLO(model_weights)
    model.train(
        data=str(data_yaml_path),
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        device=device,
        optimizer='AdamW',
        project='YOLOv11-Experiments',
        exist_ok=True,
    )
    return model


In [ ]:
# Example usage (update paths before running):
# yolo_model = train_yolo_model(
#     base_data_dir='../../data',
#     output_dir='data_yolo',
#     epochs=100,
#     imgsz=512,
#     batch=48,
#     device=0,
#     model_weights='yolo11x.pt',
# )


## RF-DETR Training

In [ ]:
import glob
import json
import math
import os
import shutil
from pathlib import Path


def setup_rfdetr_directories(base_path='/workspace/data_rf'):
    '''Ensure the RF-DETR dataset folders exist.'''
    Path(base_path, 'train').mkdir(parents=True, exist_ok=True)
    Path(base_path, 'valid').mkdir(parents=True, exist_ok=True)
    Path(base_path, 'test').mkdir(parents=True, exist_ok=True)


def copy_rfdetr_data(source_path='/workspace/data', target_path='/workspace/data_rf'):
    '''Copy raw data into the RF-DETR train/valid/test splits.'''
    shutil.copytree(f'{source_path}/TS_KS', f'{target_path}/train', dirs_exist_ok=True)
    shutil.copytree(f'{source_path}/TL_KS_BBOX', f'{target_path}/train', dirs_exist_ok=True)
    shutil.copytree(f'{source_path}/VS_KS', f'{target_path}/valid', dirs_exist_ok=True)
    shutil.copytree(f'{source_path}/VL_KS_BBOX', f'{target_path}/valid', dirs_exist_ok=True)
    shutil.copytree(f'{target_path}/valid', f'{target_path}/test', dirs_exist_ok=True)


def clean_rfdetr_annotations(target_path='/workspace/data_rf'):
    '''Remove existing COCO annotation files before regeneration.'''
    for file in glob.glob(f'{target_path}/*/_annotations.coco.json'):
        os.remove(file)


def prepare_rfdetr_dataset(
    source_path='/workspace/data',
    target_path='/workspace/data_rf',
    output_path='/workspace/rfdetr_run',
):
    '''Full dataset preparation routine for RF-DETR.'''
    setup_rfdetr_directories(target_path)
    Path(output_path).mkdir(parents=True, exist_ok=True)
    copy_rfdetr_data(source_path, target_path)
    clean_rfdetr_annotations(target_path)


def preprocess_to_coco(pathname: str):
    '''Convert VIA JSON annotations into a single COCO JSON file.'''
    data_dir = Path(pathname)
    out_path = data_dir / '_annotations.coco.json'

    images, annotations = [], []
    img_id, ann_id = 1, 1

    for jf in data_dir.glob('*.json'):
        with jf.open('r', encoding='utf-8') as f:
            raw = json.load(f)
        entry = list(raw.values())[0]
        filename = entry['filename']
        fa = entry['file_attributes']
        width, height = int(fa['img_width']), int(fa['img_height'])

        images.append({'id': img_id, 'file_name': filename, 'width': width, 'height': height})

        for region in entry['regions']:
            sa = region['shape_attributes']
            if sa['name'] == 'rect':
                x, y, w, h = map(int, (sa['x'], sa['y'], sa['width'], sa['height']))
                x, y = max(0, x), max(0, y)
                w = min(w, width - x)
                h = min(h, height - y)

                if w > 0 and h > 0:
                    annotations.append(
                        {
                            'id': ann_id,
                            'image_id': img_id,
                            'category_id': 0,
                            'bbox': [x, y, w, h],
                            'area': w * h,
                            'segmentation': [[x, y, x + w, y, x + w, y + h, x, y + h]],
                            'iscrowd': 0,
                        }
                    )
                    ann_id += 1
        img_id += 1

    coco = {
        'info': {},
        'licenses': [],
        'images': images,
        'annotations': annotations,
        'categories': [{'id': 0, 'name': 'chimney', 'supercategory': 'none'}],
    }

    out_path.write_text(json.dumps(coco, ensure_ascii=False, indent=2), encoding='utf-8')
    return len(images), len(annotations)


def validate_coco(annotation_path: str):
    '''Light-weight structural validation for the generated COCO annotations.'''
    with open(annotation_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    images = {im['id']: im for im in data['images']}

    for ann in data['annotations']:
        im = images[ann['image_id']]
        w_img, h_img = im['width'], im['height']
        x, y, w, h = ann['bbox']

        if len(ann['bbox']) != 4 or w <= 0 or h <= 0:
            return False
        if x < 0 or y < 0 or (x + w) > w_img or (y + h) > h_img:
            return False
        if any(math.isnan(v) or math.isinf(v) for v in (x, y, w, h)):
            return False

    return True


def process_annotations(base_path='/workspace/data_rf'):
    '''Generate COCO annotations for each split and report basic stats.'''
    splits = ['train', 'valid', 'test']
    stats = {}

    for split in splits:
        split_path = f'{base_path}/{split}'
        if Path(split_path).exists():
            img_count, ann_count = preprocess_to_coco(split_path)
            is_valid = validate_coco(f'{split_path}/_annotations.coco.json')
            stats[split] = {'images': img_count, 'annotations': ann_count, 'valid': is_valid}
        else:
            stats[split] = {'images': 0, 'annotations': 0, 'valid': False}

    return stats


def train_rfdetr(
    dataset_path='/workspace/data_rf',
    output_path='/workspace/rfdetr_run',
    model_type='large',
    epochs=50,
    batch_size=6,
    grad_accum_steps=4,
    lr=1e-4,
):
    '''Launch RF-DETR training.'''
    dataset_root = Path(dataset_path)
    if not dataset_root.exists():
        raise FileNotFoundError(f'Dataset path not found: {dataset_path}')

    Path(output_path).mkdir(parents=True, exist_ok=True)

    model_type_lower = model_type.lower()
    if model_type_lower == 'large':
        from rfdetr.detr import RFDETRLarge as ModelCls
    elif model_type_lower == 'base':
        from rfdetr.detr import RFDETRBase as ModelCls
    elif model_type_lower == 'small':
        from rfdetr.detr import RFDETRSmall as ModelCls
    else:
        raise ValueError(f'Unsupported model type: {model_type}')

    model = ModelCls()
    model.train(
        dataset_dir=dataset_path,
        epochs=epochs,
        batch_size=batch_size,
        grad_accum_steps=grad_accum_steps,
        lr=lr,
        output_dir=output_path,
    )
    return model


In [ ]:
# Example usage (update paths before running):
# prepare_rfdetr_dataset(
#     source_path='../../data',
#     target_path='/workspace/data_rf',
#     output_path='/workspace/rfdetr_run',
# )
#
# annotation_stats = process_annotations(base_path='/workspace/data_rf')
# print(annotation_stats)
#
# rfdetr_model = train_rfdetr(
#     dataset_path='/workspace/data_rf',
#     output_path='/workspace/rfdetr_run',
#     model_type='large',
#     epochs=50,
# )


## RT-DETR Training

In [ ]:
import json
import os
from typing import Dict, Optional

import albumentations as A
from datasets import Dataset, Image
import numpy as np
import torch
from transformers.models.auto.image_processing_auto import AutoImageProcessor
from transformers.models.auto.modeling_auto import AutoModelForObjectDetection


def load_training_components(checkpoint: str) -> Dict[str, object]:
    '''Load the RT-DETR backbone, processor, and augmentations.'''
    image_processor = AutoImageProcessor.from_pretrained(
        checkpoint,
        use_fast=True,
    )

    train_transform = A.Compose(
        [
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.GaussNoise(p=0.5),
        ],
        bbox_params=A.BboxParams(
            format='coco',
            label_fields=['category'],
            clip=True,
            min_area=25,
            min_width=1,
            min_height=1,
        ),
    )

    validation_transform = A.Compose(
        [A.NoOp()],
        bbox_params=A.BboxParams(
            format='coco',
            label_fields=['category'],
            clip=True,
            min_area=1,
            min_width=1,
            min_height=1,
        ),
    )

    id2label = {0: 'chimney'}
    label2id = {'chimney': 0}

    model = AutoModelForObjectDetection.from_pretrained(
        checkpoint,
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True,
    )

    return {
        'image_processor': image_processor,
        'train_transform': train_transform,
        'validation_transform': validation_transform,
        'model': model,
    }


def loading_data(label_path: str, image_path: str) -> Dataset:
    '''Read VIA JSON annotations into a HuggingFace dataset.'''
    json_files = [f for f in os.listdir(label_path) if f.endswith('.json')]

    combined_data = {
        'image_id': [],
        'image_name': [],
        'image': [],
        'bboxes': [],
        'category': [],
    }
    idx = 0
    for json_file in json_files:
        with open(os.path.join(label_path, json_file), 'r', encoding='utf-8') as f:
            data = json.load(f)
        image_info = list(data.values())[0]
        filename = image_info['filename']
        image_full_path = os.path.join(image_path, filename)

        combined_data['image_name'].append(os.path.splitext(filename)[0])
        combined_data['image_id'].append(idx)
        idx += 1

        bboxes = []
        labels = []
        for region in image_info['regions']:
            attrs = region['shape_attributes']
            bbox = (attrs['x'], attrs['y'], attrs['width'], attrs['height'])
            bboxes.append(bbox)
            labels.append(0)

        combined_data['bboxes'].append(bboxes)
        combined_data['category'].append(labels)
        combined_data['image'].append(image_full_path)

    dataset = Dataset.from_dict(combined_data)
    dataset = dataset.cast_column('image', Image())
    return dataset


class RTDataset(Dataset):
    '''Dataset wrapper that applies Albumentations transforms and processor formatting.'''

    def __init__(self, dataset: Dataset, image_processor, transform: Optional[A.Compose] = None):
        self.dataset = dataset
        self.image_processor = image_processor
        self.transform = transform

    @staticmethod
    def format_image_annotations_as_coco(image_id, categories, boxes):
        annotations = []
        for category, bbox in zip(categories, boxes):
            formatted = {
                'image_id': image_id,
                'category_id': category,
                'bbox': bbox,
                'iscrowd': 0,
                'area': bbox[2] * bbox[3],
            }
            annotations.append(formatted)
        return {'image_id': image_id, 'annotations': annotations}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        image_id = sample['image_id']
        image = np.array(sample['image'])
        boxes = sample['bboxes']
        categories = sample['category']

        if self.transform:
            transformed = self.transform(image=image, bboxes=boxes, category=categories)
            image = transformed['image']
            boxes = transformed['bboxes']
            categories = transformed['category']

        annotations = self.format_image_annotations_as_coco(image_id, categories, boxes)
        processed = self.image_processor(
            images=image,
            annotations=annotations,
            return_tensors='pt',
        )

        return processed


def collate_fn(batch):
    pixel_values = torch.stack([x['pixel_values'] for x in batch])
    labels = [x['labels'] for x in batch]
    return {'pixel_values': pixel_values, 'labels': labels}


In [ ]:
from transformers.training_args import TrainingArguments
from transformers.trainer import Trainer


def train_rtdetr_model(
    checkpoint='PekingU/rtdetr_r50vd',
    paths=None,
    output_dir='/workspace/RT_50',
    num_train_epochs=60,
    learning_rate=1e-4,
    warmup_steps=300,
    per_device_train_batch_size=24,
    dataloader_num_workers=2,
):
    '''Train RT-DETR without validation metrics.'''
    if paths is None:
        raise ValueError('Provide a dictionary with at least train_json and train_jpg paths.')

    components = load_training_components(checkpoint)
    image_processor = components['image_processor']
    train_transform = components['train_transform']
    model = components['model']

    train_data = loading_data(paths['train_json'], paths['train_jpg'])
    train_dataset = RTDataset(train_data, image_processor, transform=train_transform)

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,
        max_grad_norm=0.1,
        learning_rate=learning_rate,
        warmup_steps=warmup_steps,
        per_device_train_batch_size=per_device_train_batch_size,
        dataloader_num_workers=dataloader_num_workers,
        evaluation_strategy='no',
        save_strategy='epoch',
        save_total_limit=2,
        remove_unused_columns=False,
        report_to='none',
        logging_steps=50,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=collate_fn,
        processing_class=image_processor,
    )
    trainer.train()
    return trainer


In [ ]:
# Example usage (update paths before running):
# data_paths = {
#     'train_json': '/root/workspace/data/TL_KS_BBOX',
#     'train_jpg': '/root/workspace/data/TS_KS',
# }
#
# trainer = train_rtdetr_model(
#     checkpoint='PekingU/rtdetr_r50vd',
#     paths=data_paths,
#     output_dir='/workspace/RT_50',
#     num_train_epochs=60,
# )


## Training Logs

### YOLOv11 Two-Stage Training Metrics

Summary across both training stages:

| stage | epochs | best_epoch_map50 | best_map50 | best_map50_95 | best_precision | best_recall | final_map50 | final_map50_95 | final_precision | final_recall |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Stage 1 | 60 | 57 | 0.9920 | 0.8723 | 0.9863 | 0.9871 | 0.9920 | 0.8723 | 0.9774 | 0.9871 |
| Stage 2 | 100 | 99 | 0.9931 | 0.8846 | 0.9875 | 0.9889 | 0.9930 | 0.8846 | 0.9849 | 0.9869 |

**Stage 1 full metrics**

| epoch | time | train/box_loss | train/cls_loss | train/dfl_loss | metrics/precision(B) | metrics/recall(B) | metrics/mAP50(B) | metrics/mAP50-95(B) | val/box_loss | val/cls_loss | val/dfl_loss | lr/pg0 | lr/pg1 | lr/pg2 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | 67.7335 | 1.3625 | 1.3654 | 1.2014 | 0.1097 | 0.4372 | 0.0706 | 0.0364 | 1.7381 | 12.7079 | 1.6670 | 0.0672 | 0.0003 | 0.0003 |
| 2 | 127.0630 | 1.3856 | 1.1802 | 1.2321 | 0.3861 | 0.5923 | 0.3378 | 0.2004 | 1.3935 | 5.8620 | 1.4047 | 0.0342 | 0.0007 | 0.0007 |
| 3 | 186.6020 | 1.3964 | 1.1912 | 1.2433 | 0.6511 | 0.5832 | 0.6365 | 0.3952 | 1.3824 | 4.9355 | 1.3553 | 0.0012 | 0.0010 | 0.0010 |
| 4 | 245.7150 | 1.3174 | 1.0740 | 1.2027 | 0.8163 | 0.7322 | 0.8176 | 0.5434 | 1.1700 | 1.1336 | 1.1943 | 0.0010 | 0.0010 | 0.0010 |
| 5 | 305.0380 | 1.2401 | 0.9533 | 1.1624 | 0.8492 | 0.8091 | 0.8805 | 0.5967 | 1.1316 | 1.1372 | 1.1428 | 0.0009 | 0.0009 | 0.0009 |
| 6 | 364.2660 | 1.1742 | 0.8617 | 1.1312 | 0.8400 | 0.8102 | 0.8770 | 0.6049 | 1.0869 | 0.8221 | 1.1408 | 0.0009 | 0.0009 | 0.0009 |
| 7 | 423.5520 | 1.1251 | 0.8215 | 1.1069 | 0.8837 | 0.8625 | 0.9218 | 0.6490 | 1.0391 | 0.9067 | 1.1004 | 0.0009 | 0.0009 | 0.0009 |
| 8 | 482.8960 | 1.0960 | 0.7864 | 1.0870 | 0.8860 | 0.8601 | 0.9258 | 0.6726 | 0.9967 | 0.8655 | 1.0791 | 0.0009 | 0.0009 | 0.0009 |
| 9 | 542.1330 | 1.0586 | 0.7398 | 1.0625 | 0.9307 | 0.9024 | 0.9590 | 0.7183 | 0.9294 | 0.5876 | 1.0456 | 0.0009 | 0.0009 | 0.0009 |
| 10 | 601.3940 | 1.0286 | 0.7160 | 1.0551 | 0.8912 | 0.9235 | 0.9574 | 0.7168 | 0.9216 | 0.5689 | 1.0362 | 0.0009 | 0.0009 | 0.0009 |
| 11 | 660.4490 | 1.0140 | 0.6892 | 1.0401 | 0.9448 | 0.9054 | 0.9666 | 0.7212 | 0.9150 | 0.5635 | 1.0260 | 0.0008 | 0.0008 | 0.0008 |
| 12 | 719.7560 | 0.9955 | 0.6796 | 1.0321 | 0.9237 | 0.9085 | 0.9573 | 0.7039 | 0.9234 | 0.6780 | 1.0281 | 0.0008 | 0.0008 | 0.0008 |
| 13 | 778.8300 | 0.9866 | 0.6651 | 1.0293 | 0.9374 | 0.9065 | 0.9638 | 0.7216 | 0.9137 | 0.5587 | 1.0169 | 0.0008 | 0.0008 | 0.0008 |
| 14 | 838.0600 | 0.9718 | 0.6528 | 1.0221 | 0.9243 | 0.9486 | 0.9712 | 0.7477 | 0.8613 | 0.5300 | 0.9859 | 0.0008 | 0.0008 | 0.0008 |
| 15 | 897.3290 | 0.9490 | 0.6324 | 1.0115 | 0.9357 | 0.9054 | 0.9642 | 0.7321 | 0.8832 | 0.5473 | 1.0063 | 0.0008 | 0.0008 | 0.0008 |
| 16 | 956.3930 | 0.9339 | 0.6200 | 1.0030 | 0.9254 | 0.9425 | 0.9767 | 0.7582 | 0.8611 | 0.4895 | 0.9927 | 0.0008 | 0.0008 | 0.0008 |
| 17 | 1015.5500 | 0.9319 | 0.6067 | 1.0036 | 0.9554 | 0.9039 | 0.9665 | 0.7620 | 0.8210 | 0.5044 | 0.9694 | 0.0007 | 0.0007 | 0.0007 |
| 18 | 1074.8300 | 0.9177 | 0.5998 | 0.9938 | 0.9336 | 0.9334 | 0.9664 | 0.7331 | 0.8864 | 0.5435 | 1.0063 | 0.0007 | 0.0007 | 0.0007 |
| 19 | 1133.9000 | 0.9008 | 0.5806 | 0.9897 | 0.9477 | 0.9594 | 0.9822 | 0.7796 | 0.8005 | 0.4504 | 0.9538 | 0.0007 | 0.0007 | 0.0007 |
| 20 | 1193.1700 | 0.8865 | 0.5647 | 0.9831 | 0.9472 | 0.9626 | 0.9843 | 0.7817 | 0.7953 | 0.4340 | 0.9539 | 0.0007 | 0.0007 | 0.0007 |
| 21 | 1252.3900 | 0.8829 | 0.5629 | 0.9801 | 0.9513 | 0.9448 | 0.9831 | 0.7756 | 0.8180 | 0.4447 | 0.9597 | 0.0007 | 0.0007 | 0.0007 |
| 22 | 1311.4400 | 0.8784 | 0.5552 | 0.9791 | 0.9395 | 0.9607 | 0.9835 | 0.7816 | 0.7835 | 0.4384 | 0.9512 | 0.0007 | 0.0007 | 0.0007 |
| 23 | 1370.4100 | 0.8577 | 0.5448 | 0.9690 | 0.9619 | 0.9493 | 0.9843 | 0.7938 | 0.7760 | 0.4280 | 0.9430 | 0.0006 | 0.0006 | 0.0006 |
| 24 | 1429.7700 | 0.8525 | 0.5319 | 0.9658 | 0.9640 | 0.9493 | 0.9862 | 0.8018 | 0.7708 | 0.4143 | 0.9409 | 0.0006 | 0.0006 | 0.0006 |
| 25 | 1488.8800 | 0.8447 | 0.5264 | 0.9622 | 0.9528 | 0.9622 | 0.9854 | 0.7917 | 0.7623 | 0.4149 | 0.9386 | 0.0006 | 0.0006 | 0.0006 |
| 26 | 1547.9300 | 0.8336 | 0.5193 | 0.9558 | 0.9562 | 0.9697 | 0.9835 | 0.8045 | 0.7464 | 0.4034 | 0.9353 | 0.0006 | 0.0006 | 0.0006 |
| 27 | 1607.1600 | 0.8246 | 0.5138 | 0.9551 | 0.9589 | 0.9660 | 0.9872 | 0.7935 | 0.7616 | 0.4034 | 0.9352 | 0.0006 | 0.0006 | 0.0006 |
| 28 | 1666.1900 | 0.8136 | 0.5051 | 0.9498 | 0.9600 | 0.9728 | 0.9868 | 0.8073 | 0.7403 | 0.3823 | 0.9283 | 0.0006 | 0.0006 | 0.0006 |
| 29 | 1725.3100 | 0.8096 | 0.5028 | 0.9472 | 0.9664 | 0.9614 | 0.9876 | 0.8105 | 0.7335 | 0.3761 | 0.9300 | 0.0005 | 0.0005 | 0.0005 |
| 30 | 1784.5600 | 0.8046 | 0.4949 | 0.9434 | 0.9544 | 0.9644 | 0.9862 | 0.7923 | 0.7689 | 0.3904 | 0.9319 | 0.0005 | 0.0005 | 0.0005 |
| 31 | 1843.5800 | 0.8007 | 0.4881 | 0.9486 | 0.9661 | 0.9705 | 0.9889 | 0.8149 | 0.7209 | 0.3688 | 0.9151 | 0.0005 | 0.0005 | 0.0005 |
| 32 | 1902.6600 | 0.7876 | 0.4793 | 0.9391 | 0.9667 | 0.9660 | 0.9898 | 0.8221 | 0.7224 | 0.3641 | 0.9198 | 0.0005 | 0.0005 | 0.0005 |
| 33 | 1961.9200 | 0.7842 | 0.4743 | 0.9380 | 0.9726 | 0.9666 | 0.9893 | 0.8209 | 0.7015 | 0.3583 | 0.9118 | 0.0005 | 0.0005 | 0.0005 |
| 34 | 2020.9700 | 0.7713 | 0.4711 | 0.9331 | 0.9666 | 0.9788 | 0.9902 | 0.8270 | 0.6959 | 0.3548 | 0.9062 | 0.0005 | 0.0005 | 0.0005 |
| 35 | 2080.3100 | 0.7737 | 0.4623 | 0.9331 | 0.9676 | 0.9697 | 0.9898 | 0.8137 | 0.7177 | 0.3630 | 0.9086 | 0.0004 | 0.0004 | 0.0004 |
| 36 | 2139.4300 | 0.7604 | 0.4574 | 0.9333 | 0.9647 | 0.9644 | 0.9878 | 0.8230 | 0.7047 | 0.3542 | 0.9092 | 0.0004 | 0.0004 | 0.0004 |
| 37 | 2198.4300 | 0.7494 | 0.4488 | 0.9278 | 0.9679 | 0.9808 | 0.9896 | 0.8410 | 0.6797 | 0.3405 | 0.8996 | 0.0004 | 0.0004 | 0.0004 |
| 38 | 2257.7500 | 0.7376 | 0.4396 | 0.9238 | 0.9653 | 0.9796 | 0.9892 | 0.8269 | 0.6942 | 0.3464 | 0.9039 | 0.0004 | 0.0004 | 0.0004 |
| 39 | 2316.8000 | 0.7354 | 0.4318 | 0.9234 | 0.9685 | 0.9768 | 0.9891 | 0.8431 | 0.6703 | 0.3336 | 0.8924 | 0.0004 | 0.0004 | 0.0004 |
| 40 | 2376.1200 | 0.7319 | 0.4306 | 0.9181 | 0.9740 | 0.9773 | 0.9905 | 0.8333 | 0.6745 | 0.3407 | 0.8913 | 0.0004 | 0.0004 | 0.0004 |
| 41 | 2435.1500 | 0.7284 | 0.4256 | 0.9200 | 0.9744 | 0.9776 | 0.9903 | 0.8418 | 0.6596 | 0.3276 | 0.8864 | 0.0003 | 0.0003 | 0.0003 |
| 42 | 2494.1800 | 0.7176 | 0.4208 | 0.9162 | 0.9714 | 0.9788 | 0.9902 | 0.8491 | 0.6426 | 0.3186 | 0.8803 | 0.0003 | 0.0003 | 0.0003 |
| 43 | 2553.4900 | 0.7076 | 0.4147 | 0.9073 | 0.9602 | 0.9856 | 0.9895 | 0.8433 | 0.6533 | 0.3225 | 0.8830 | 0.0003 | 0.0003 | 0.0003 |
| 44 | 2612.5700 | 0.7030 | 0.4100 | 0.9121 | 0.9788 | 0.9769 | 0.9906 | 0.8485 | 0.6411 | 0.3143 | 0.8799 | 0.0003 | 0.0003 | 0.0003 |
| 45 | 2671.6100 | 0.7052 | 0.4133 | 0.9110 | 0.9771 | 0.9697 | 0.9908 | 0.8487 | 0.6475 | 0.3180 | 0.8844 | 0.0003 | 0.0003 | 0.0003 |
| 46 | 2730.8100 | 0.6887 | 0.3993 | 0.9055 | 0.9780 | 0.9756 | 0.9909 | 0.8520 | 0.6374 | 0.3142 | 0.8804 | 0.0003 | 0.0003 | 0.0003 |
| 47 | 2790.1200 | 0.6848 | 0.3986 | 0.9048 | 0.9708 | 0.9818 | 0.9906 | 0.8522 | 0.6352 | 0.3101 | 0.8750 | 0.0002 | 0.0002 | 0.0002 |
| 48 | 2849.5500 | 0.6812 | 0.3945 | 0.9050 | 0.9744 | 0.9806 | 0.9908 | 0.8567 | 0.6316 | 0.3074 | 0.8734 | 0.0002 | 0.0002 | 0.0002 |
| 49 | 2908.8300 | 0.6769 | 0.3938 | 0.9011 | 0.9766 | 0.9767 | 0.9916 | 0.8537 | 0.6291 | 0.3088 | 0.8710 | 0.0002 | 0.0002 | 0.0002 |
| 50 | 2967.9400 | 0.6678 | 0.3918 | 0.8980 | 0.9774 | 0.9836 | 0.9908 | 0.8549 | 0.6256 | 0.3028 | 0.8717 | 0.0002 | 0.0002 | 0.0002 |
| 51 | 3027.8600 | 0.6209 | 0.3333 | 0.8919 | 0.9752 | 0.9750 | 0.9910 | 0.8598 | 0.6279 | 0.3058 | 0.8717 | 0.0002 | 0.0002 | 0.0002 |
| 52 | 3087.2500 | 0.6089 | 0.3229 | 0.8867 | 0.9766 | 0.9795 | 0.9911 | 0.8634 | 0.6223 | 0.3065 | 0.8704 | 0.0002 | 0.0002 | 0.0002 |
| 53 | 3146.6200 | 0.6086 | 0.3214 | 0.8845 | 0.9734 | 0.9856 | 0.9913 | 0.8661 | 0.6119 | 0.2980 | 0.8668 | 0.0001 | 0.0001 | 0.0001 |
| 54 | 3206.1500 | 0.5952 | 0.3133 | 0.8818 | 0.9776 | 0.9826 | 0.9915 | 0.8689 | 0.6122 | 0.2949 | 0.8682 | 0.0001 | 0.0001 | 0.0001 |
| 55 | 3265.4400 | 0.5906 | 0.3122 | 0.8808 | 0.9818 | 0.9803 | 0.9918 | 0.8693 | 0.6066 | 0.2904 | 0.8662 | 0.0001 | 0.0001 | 0.0001 |
| 56 | 3324.9600 | 0.5870 | 0.3087 | 0.8829 | 0.9863 | 0.9773 | 0.9918 | 0.8700 | 0.6049 | 0.2890 | 0.8653 | 0.0001 | 0.0001 | 0.0001 |
| 57 | 3384.6000 | 0.5784 | 0.3036 | 0.8779 | 0.9833 | 0.9811 | 0.9920 | 0.8712 | 0.5993 | 0.2870 | 0.8648 | 0.0001 | 0.0001 | 0.0001 |
| 58 | 3443.9400 | 0.5747 | 0.2965 | 0.8742 | 0.9838 | 0.9788 | 0.9919 | 0.8710 | 0.5999 | 0.2860 | 0.8645 | 0.0001 | 0.0001 | 0.0001 |
| 59 | 3503.1200 | 0.5692 | 0.2946 | 0.8756 | 0.9811 | 0.9828 | 0.9920 | 0.8716 | 0.5995 | 0.2837 | 0.8645 | 0 | 0 | 0 |
| 60 | 3562.4000 | 0.5627 | 0.2911 | 0.8715 | 0.9774 | 0.9871 | 0.9920 | 0.8723 | 0.5986 | 0.2838 | 0.8644 | 0 | 0 | 0 |

**Stage 2 full metrics**

| epoch | time | train/box_loss | train/cls_loss | train/dfl_loss | metrics/precision(B) | metrics/recall(B) | metrics/mAP50(B) | metrics/mAP50-95(B) | val/box_loss | val/cls_loss | val/dfl_loss | lr/pg0 | lr/pg1 | lr/pg2 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | 67.6009 | 0.7019 | 0.4259 | 0.9048 | 0.9651 | 0.9633 | 0.9865 | 0.8200 | 0.6917 | 0.3682 | 0.9050 | 0.0669 | 0 | 0 |
| 2 | 126.7120 | 0.6724 | 0.3963 | 0.9022 | 0.9620 | 0.9788 | 0.9891 | 0.7978 | 0.7609 | 0.3676 | 0.9369 | 0.0336 | 0.0001 | 0.0001 |
| 3 | 185.9230 | 0.6553 | 0.3741 | 0.8930 | 0.9807 | 0.9811 | 0.9918 | 0.8628 | 0.6117 | 0.2976 | 0.8683 | 0.0003 | 0.0001 | 0.0001 |
| 4 | 245.3760 | 0.6388 | 0.3599 | 0.8876 | 0.9789 | 0.9841 | 0.9921 | 0.8676 | 0.6079 | 0.2946 | 0.8671 | 0.0001 | 0.0001 | 0.0001 |
| 5 | 304.7570 | 0.6328 | 0.3596 | 0.8912 | 0.9804 | 0.9835 | 0.9919 | 0.8614 | 0.6213 | 0.2933 | 0.8713 | 0.0001 | 0.0001 | 0.0001 |
| 6 | 363.9780 | 0.6272 | 0.3554 | 0.8872 | 0.9788 | 0.9804 | 0.9918 | 0.8561 | 0.6247 | 0.3002 | 0.8717 | 0.0001 | 0.0001 | 0.0001 |
| 7 | 423.2450 | 0.6255 | 0.3571 | 0.8863 | 0.9811 | 0.9815 | 0.9917 | 0.8632 | 0.6067 | 0.2968 | 0.8696 | 0.0001 | 0.0001 | 0.0001 |
| 8 | 482.4600 | 0.6227 | 0.3554 | 0.8856 | 0.9818 | 0.9820 | 0.9921 | 0.8642 | 0.6008 | 0.2926 | 0.8678 | 0.0001 | 0.0001 | 0.0001 |
| 9 | 541.7170 | 0.6171 | 0.3479 | 0.8820 | 0.9734 | 0.9818 | 0.9919 | 0.8664 | 0.6102 | 0.2938 | 0.8683 | 0.0001 | 0.0001 | 0.0001 |
| 10 | 600.9260 | 0.6194 | 0.3546 | 0.8852 | 0.9773 | 0.9841 | 0.9917 | 0.8666 | 0.6031 | 0.2906 | 0.8670 | 0.0001 | 0.0001 | 0.0001 |
| 11 | 660.1400 | 0.6168 | 0.3446 | 0.8826 | 0.9804 | 0.9830 | 0.9919 | 0.8699 | 0.5978 | 0.2884 | 0.8686 | 0.0001 | 0.0001 | 0.0001 |
| 12 | 719.6230 | 0.6070 | 0.3444 | 0.8801 | 0.9793 | 0.9788 | 0.9920 | 0.8658 | 0.6000 | 0.2877 | 0.8649 | 0.0001 | 0.0001 | 0.0001 |
| 13 | 778.8270 | 0.6124 | 0.3431 | 0.8812 | 0.9822 | 0.9811 | 0.9919 | 0.8627 | 0.6041 | 0.2872 | 0.8673 | 0.0001 | 0.0001 | 0.0001 |
| 14 | 838.0290 | 0.6113 | 0.3437 | 0.8830 | 0.9762 | 0.9856 | 0.9921 | 0.8674 | 0.5982 | 0.2873 | 0.8648 | 0.0001 | 0.0001 | 0.0001 |
| 15 | 897.2780 | 0.6022 | 0.3415 | 0.8792 | 0.9804 | 0.9826 | 0.9919 | 0.8653 | 0.6106 | 0.2928 | 0.8663 | 0.0001 | 0.0001 | 0.0001 |
| 16 | 956.4950 | 0.6034 | 0.3429 | 0.8807 | 0.9833 | 0.9794 | 0.9921 | 0.8701 | 0.5983 | 0.2886 | 0.8633 | 0.0001 | 0.0001 | 0.0001 |
| 17 | 1015.8900 | 0.5982 | 0.3388 | 0.8804 | 0.9806 | 0.9856 | 0.9919 | 0.8685 | 0.6006 | 0.2865 | 0.8639 | 0.0001 | 0.0001 | 0.0001 |
| 18 | 1075.1600 | 0.6047 | 0.3401 | 0.8789 | 0.9814 | 0.9841 | 0.9920 | 0.8634 | 0.6029 | 0.2864 | 0.8659 | 0.0001 | 0.0001 | 0.0001 |
| 19 | 1134.4000 | 0.5935 | 0.3352 | 0.8765 | 0.9811 | 0.9838 | 0.9926 | 0.8720 | 0.5939 | 0.2882 | 0.8614 | 0.0001 | 0.0001 | 0.0001 |
| 20 | 1193.8600 | 0.5900 | 0.3326 | 0.8769 | 0.9841 | 0.9809 | 0.9921 | 0.8652 | 0.5920 | 0.2871 | 0.8611 | 0.0001 | 0.0001 | 0.0001 |
| 21 | 1253.1300 | 0.5912 | 0.3349 | 0.8776 | 0.9790 | 0.9851 | 0.9928 | 0.8693 | 0.5982 | 0.2872 | 0.8621 | 0.0001 | 0.0001 | 0.0001 |
| 22 | 1312.3600 | 0.5926 | 0.3325 | 0.8767 | 0.9800 | 0.9864 | 0.9922 | 0.8673 | 0.5907 | 0.2832 | 0.8616 | 0.0001 | 0.0001 | 0.0001 |
| 23 | 1371.6000 | 0.5824 | 0.3289 | 0.8743 | 0.9760 | 0.9850 | 0.9922 | 0.8677 | 0.5943 | 0.2875 | 0.8632 | 0.0001 | 0.0001 | 0.0001 |
| 24 | 1430.8200 | 0.5883 | 0.3309 | 0.8769 | 0.9793 | 0.9834 | 0.9920 | 0.8668 | 0.5962 | 0.2874 | 0.8628 | 0.0001 | 0.0001 | 0.0001 |
| 25 | 1490.0800 | 0.5833 | 0.3262 | 0.8740 | 0.9819 | 0.9858 | 0.9928 | 0.8678 | 0.5896 | 0.2875 | 0.8604 | 0.0001 | 0.0001 | 0.0001 |
| 26 | 1549.3600 | 0.5794 | 0.3250 | 0.8745 | 0.9841 | 0.9827 | 0.9930 | 0.8731 | 0.5853 | 0.2828 | 0.8603 | 0.0001 | 0.0001 | 0.0001 |
| 27 | 1608.8000 | 0.5727 | 0.3255 | 0.8704 | 0.9812 | 0.9834 | 0.9928 | 0.8684 | 0.5860 | 0.2848 | 0.8599 | 0.0001 | 0.0001 | 0.0001 |
| 28 | 1667.9100 | 0.5740 | 0.3230 | 0.8730 | 0.9843 | 0.9834 | 0.9927 | 0.8726 | 0.5871 | 0.2856 | 0.8601 | 0.0001 | 0.0001 | 0.0001 |
| 29 | 1727.1400 | 0.5782 | 0.3244 | 0.8742 | 0.9776 | 0.9811 | 0.9918 | 0.8698 | 0.5856 | 0.2846 | 0.8595 | 0.0001 | 0.0001 | 0.0001 |
| 30 | 1786.2300 | 0.5730 | 0.3209 | 0.8711 | 0.9761 | 0.9871 | 0.9922 | 0.8700 | 0.5944 | 0.2864 | 0.8621 | 0.0001 | 0.0001 | 0.0001 |
| 31 | 1845.4900 | 0.5750 | 0.3244 | 0.8744 | 0.9811 | 0.9841 | 0.9919 | 0.8734 | 0.5842 | 0.2840 | 0.8584 | 0.0001 | 0.0001 | 0.0001 |
| 32 | 1904.8600 | 0.5692 | 0.3195 | 0.8702 | 0.9826 | 0.9812 | 0.9921 | 0.8734 | 0.5817 | 0.2814 | 0.8598 | 0.0001 | 0.0001 | 0.0001 |
| 33 | 1964.3200 | 0.5685 | 0.3169 | 0.8706 | 0.9787 | 0.9834 | 0.9918 | 0.8728 | 0.5785 | 0.2850 | 0.8574 | 0.0001 | 0.0001 | 0.0001 |
| 34 | 2023.5400 | 0.5644 | 0.3165 | 0.8676 | 0.9829 | 0.9826 | 0.9921 | 0.8774 | 0.5771 | 0.2810 | 0.8575 | 0.0001 | 0.0001 | 0.0001 |
| 35 | 2082.9100 | 0.5694 | 0.3192 | 0.8706 | 0.9804 | 0.9832 | 0.9922 | 0.8754 | 0.5792 | 0.2808 | 0.8577 | 0.0001 | 0.0001 | 0.0001 |
| 36 | 2142.1500 | 0.5593 | 0.3132 | 0.8709 | 0.9764 | 0.9879 | 0.9920 | 0.8745 | 0.5785 | 0.2800 | 0.8572 | 0.0001 | 0.0001 | 0.0001 |
| 37 | 2201.4200 | 0.5608 | 0.3121 | 0.8707 | 0.9808 | 0.9834 | 0.9920 | 0.8742 | 0.5799 | 0.2821 | 0.8584 | 0.0001 | 0.0001 | 0.0001 |
| 38 | 2260.7000 | 0.5519 | 0.3112 | 0.8681 | 0.9783 | 0.9871 | 0.9928 | 0.8749 | 0.5804 | 0.2828 | 0.8583 | 0.0001 | 0.0001 | 0.0001 |
| 39 | 2319.9400 | 0.5504 | 0.3070 | 0.8683 | 0.9811 | 0.9840 | 0.9919 | 0.8766 | 0.5750 | 0.2786 | 0.8576 | 0.0001 | 0.0001 | 0.0001 |
| 40 | 2379.2000 | 0.5517 | 0.3087 | 0.8652 | 0.9782 | 0.9868 | 0.9917 | 0.8744 | 0.5755 | 0.2796 | 0.8573 | 0.0001 | 0.0001 | 0.0001 |
| 41 | 2438.4700 | 0.5504 | 0.3072 | 0.8675 | 0.9798 | 0.9841 | 0.9918 | 0.8759 | 0.5738 | 0.2813 | 0.8566 | 0.0001 | 0.0001 | 0.0001 |
| 42 | 2497.7500 | 0.5515 | 0.3071 | 0.8683 | 0.9825 | 0.9818 | 0.9919 | 0.8736 | 0.5734 | 0.2819 | 0.8562 | 0.0001 | 0.0001 | 0.0001 |
| 43 | 2556.8500 | 0.5444 | 0.3034 | 0.8621 | 0.9826 | 0.9802 | 0.9920 | 0.8773 | 0.5692 | 0.2811 | 0.8551 | 0.0001 | 0.0001 | 0.0001 |
| 44 | 2616.0300 | 0.5440 | 0.3047 | 0.8671 | 0.9837 | 0.9826 | 0.9927 | 0.8759 | 0.5713 | 0.2819 | 0.8559 | 0.0001 | 0.0001 | 0.0001 |
| 45 | 2675.3300 | 0.5527 | 0.3139 | 0.8665 | 0.9826 | 0.9852 | 0.9918 | 0.8774 | 0.5708 | 0.2785 | 0.8555 | 0.0001 | 0.0001 | 0.0001 |
| 46 | 2734.6500 | 0.5433 | 0.3032 | 0.8644 | 0.9818 | 0.9871 | 0.9919 | 0.8753 | 0.5726 | 0.2814 | 0.8551 | 0.0001 | 0.0001 | 0.0001 |
| 47 | 2793.9600 | 0.5440 | 0.3063 | 0.8648 | 0.9831 | 0.9841 | 0.9919 | 0.8731 | 0.5728 | 0.2802 | 0.8567 | 0.0001 | 0.0001 | 0.0001 |
| 48 | 2853.2300 | 0.5439 | 0.3038 | 0.8669 | 0.9827 | 0.9858 | 0.9921 | 0.8734 | 0.5741 | 0.2784 | 0.8556 | 0.0001 | 0.0001 | 0.0001 |
| 50 | 2971.5900 | 0.5471 | 0.3108 | 0.8644 | 0.9812 | 0.9871 | 0.9918 | 0.8755 | 0.5721 | 0.2798 | 0.8549 | 0.0001 | 0.0001 | 0.0001 |
| 51 | 3030.7900 | 0.5760 | 0.3174 | 0.8777 | 0.9812 | 0.9856 | 0.9921 | 0.8765 | 0.5700 | 0.2775 | 0.8541 | 0.0001 | 0.0001 | 0.0001 |
| 52 | 3090.1100 | 0.5759 | 0.3176 | 0.8756 | 0.9835 | 0.9856 | 0.9920 | 0.8774 | 0.5698 | 0.2745 | 0.8540 | 0 | 0 | 0 |
| 53 | 3149.5200 | 0.5736 | 0.3222 | 0.8723 | 0.9834 | 0.9866 | 0.9920 | 0.8780 | 0.5681 | 0.2727 | 0.8539 | 0 | 0 | 0 |
| 54 | 3208.9100 | 0.5706 | 0.3164 | 0.8746 | 0.9819 | 0.9869 | 0.9923 | 0.8767 | 0.5680 | 0.2709 | 0.8542 | 0 | 0 | 0 |
| 55 | 3268.2000 | 0.5737 | 0.3182 | 0.8750 | 0.9820 | 0.9887 | 0.9924 | 0.8755 | 0.5659 | 0.2707 | 0.8538 | 0 | 0 | 0 |
| 56 | 3327.4600 | 0.5683 | 0.3176 | 0.8729 | 0.9798 | 0.9889 | 0.9926 | 0.8768 | 0.5666 | 0.2699 | 0.8540 | 0 | 0 | 0 |
| 57 | 3386.8000 | 0.5702 | 0.3178 | 0.8737 | 0.9827 | 0.9880 | 0.9923 | 0.8764 | 0.5656 | 0.2694 | 0.8537 | 0 | 0 | 0 |
| 58 | 3446.1600 | 0.5733 | 0.3198 | 0.8719 | 0.9812 | 0.9857 | 0.9922 | 0.8764 | 0.5656 | 0.2707 | 0.8537 | 0 | 0 | 0 |
| 59 | 3505.3200 | 0.5718 | 0.3166 | 0.8720 | 0.9841 | 0.9851 | 0.9923 | 0.8790 | 0.5629 | 0.2698 | 0.8527 | 0 | 0 | 0 |
| 60 | 3564.7800 | 0.5784 | 0.3233 | 0.8780 | 0.9850 | 0.9849 | 0.9924 | 0.8775 | 0.5646 | 0.2701 | 0.8531 | 0 | 0 | 0 |
| 61 | 3624.0500 | 0.5767 | 0.3168 | 0.8745 | 0.9856 | 0.9839 | 0.9923 | 0.8764 | 0.5644 | 0.2695 | 0.8532 | 0 | 0 | 0 |
| 62 | 3683.3600 | 0.5689 | 0.3179 | 0.8714 | 0.9848 | 0.9856 | 0.9923 | 0.8768 | 0.5651 | 0.2695 | 0.8530 | 0 | 0 | 0 |
| 63 | 3742.4900 | 0.5717 | 0.3177 | 0.8736 | 0.9858 | 0.9841 | 0.9930 | 0.8780 | 0.5653 | 0.2681 | 0.8528 | 0 | 0 | 0 |
| 64 | 3801.7800 | 0.5694 | 0.3123 | 0.8698 | 0.9857 | 0.9826 | 0.9922 | 0.8800 | 0.5631 | 0.2675 | 0.8525 | 0 | 0 | 0 |
| 65 | 3861.2800 | 0.5630 | 0.3143 | 0.8692 | 0.9857 | 0.9841 | 0.9922 | 0.8807 | 0.5618 | 0.2672 | 0.8523 | 0 | 0 | 0 |
| 66 | 3920.7800 | 0.5662 | 0.3113 | 0.8738 | 0.9859 | 0.9864 | 0.9921 | 0.8801 | 0.5617 | 0.2671 | 0.8525 | 0 | 0 | 0 |
| 67 | 3980.0400 | 0.5645 | 0.3154 | 0.8708 | 0.9865 | 0.9879 | 0.9929 | 0.8808 | 0.5619 | 0.2657 | 0.8526 | 0 | 0 | 0 |
| 68 | 4039.5800 | 0.5604 | 0.3117 | 0.8710 | 0.9875 | 0.9871 | 0.9928 | 0.8791 | 0.5625 | 0.2653 | 0.8524 | 0 | 0 | 0 |
| 69 | 4098.9500 | 0.5607 | 0.3095 | 0.8689 | 0.9864 | 0.9872 | 0.9928 | 0.8786 | 0.5633 | 0.2670 | 0.8528 | 0 | 0 | 0 |
| 70 | 4158.3000 | 0.5649 | 0.3110 | 0.8694 | 0.9856 | 0.9871 | 0.9928 | 0.8776 | 0.5630 | 0.2681 | 0.8530 | 0 | 0 | 0 |
| 71 | 4217.6100 | 0.5554 | 0.3091 | 0.8660 | 0.9850 | 0.9856 | 0.9928 | 0.8771 | 0.5625 | 0.2671 | 0.8525 | 0 | 0 | 0 |
| 72 | 4276.9100 | 0.5600 | 0.3078 | 0.8681 | 0.9841 | 0.9852 | 0.9929 | 0.8770 | 0.5619 | 0.2669 | 0.8525 | 0 | 0 | 0 |
| 73 | 4336.1400 | 0.5590 | 0.3088 | 0.8724 | 0.9841 | 0.9851 | 0.9928 | 0.8777 | 0.5623 | 0.2667 | 0.8527 | 0 | 0 | 0 |
| 74 | 4395.3900 | 0.5593 | 0.3108 | 0.8709 | 0.9834 | 0.9858 | 0.9928 | 0.8770 | 0.5616 | 0.2675 | 0.8527 | 0 | 0 | 0 |
| 75 | 4454.6700 | 0.5560 | 0.3076 | 0.8695 | 0.9841 | 0.9851 | 0.9928 | 0.8776 | 0.5613 | 0.2672 | 0.8528 | 0 | 0 | 0 |
| 76 | 4513.7800 | 0.5598 | 0.3094 | 0.8683 | 0.9844 | 0.9849 | 0.9927 | 0.8781 | 0.5611 | 0.2672 | 0.8526 | 0 | 0 | 0 |
| 77 | 4573.0800 | 0.5555 | 0.3058 | 0.8674 | 0.9855 | 0.9849 | 0.9928 | 0.8786 | 0.5612 | 0.2675 | 0.8524 | 0 | 0 | 0 |
| 78 | 4632.2900 | 0.5559 | 0.3061 | 0.8681 | 0.9856 | 0.9841 | 0.9922 | 0.8794 | 0.5612 | 0.2675 | 0.8525 | 0 | 0 | 0 |
| 79 | 4691.6400 | 0.5452 | 0.3006 | 0.8658 | 0.9851 | 0.9841 | 0.9922 | 0.8797 | 0.5609 | 0.2675 | 0.8523 | 0 | 0 | 0 |
| 80 | 4750.8900 | 0.5585 | 0.3081 | 0.8671 | 0.9843 | 0.9841 | 0.9922 | 0.8793 | 0.5605 | 0.2683 | 0.8522 | 0 | 0 | 0 |
| 81 | 4809.9800 | 0.5552 | 0.3058 | 0.8664 | 0.9841 | 0.9854 | 0.9921 | 0.8797 | 0.5598 | 0.2680 | 0.8519 | 0 | 0 | 0 |
| 82 | 4869.2200 | 0.5509 | 0.3011 | 0.8651 | 0.9847 | 0.9849 | 0.9921 | 0.8801 | 0.5590 | 0.2680 | 0.8518 | 0 | 0 | 0 |
| 83 | 4928.4800 | 0.5535 | 0.3052 | 0.8708 | 0.9845 | 0.9856 | 0.9921 | 0.8809 | 0.5585 | 0.2677 | 0.8516 | 0 | 0 | 0 |
| 84 | 4987.9600 | 0.5533 | 0.3040 | 0.8682 | 0.9849 | 0.9855 | 0.9921 | 0.8805 | 0.5581 | 0.2671 | 0.8516 | 0 | 0 | 0 |
| 85 | 5047.2900 | 0.5507 | 0.3029 | 0.8646 | 0.9849 | 0.9854 | 0.9921 | 0.8809 | 0.5579 | 0.2678 | 0.8516 | 0 | 0 | 0 |
| 86 | 5106.5700 | 0.5518 | 0.3021 | 0.8684 | 0.9849 | 0.9859 | 0.9922 | 0.8808 | 0.5577 | 0.2673 | 0.8517 | 0 | 0 | 0 |
| 87 | 5165.8000 | 0.5520 | 0.3049 | 0.8665 | 0.9849 | 0.9861 | 0.9922 | 0.8811 | 0.5573 | 0.2672 | 0.8516 | 0 | 0 | 0 |
| 88 | 5225.2900 | 0.5507 | 0.3035 | 0.8643 | 0.9841 | 0.9864 | 0.9922 | 0.8815 | 0.5569 | 0.2668 | 0.8515 | 0 | 0 | 0 |
| 89 | 5284.7800 | 0.5528 | 0.3057 | 0.8672 | 0.9838 | 0.9864 | 0.9922 | 0.8811 | 0.5571 | 0.2668 | 0.8516 | 0 | 0 | 0 |
| 90 | 5344.0900 | 0.5514 | 0.3065 | 0.8654 | 0.9834 | 0.9864 | 0.9923 | 0.8815 | 0.5572 | 0.2662 | 0.8517 | 0 | 0 | 0 |
| 91 | 5404.0200 | 0.5122 | 0.2668 | 0.8539 | 0.9827 | 0.9867 | 0.9923 | 0.8819 | 0.5575 | 0.2661 | 0.8517 | 0 | 0 | 0 |
| 92 | 5463.3400 | 0.5075 | 0.2599 | 0.8551 | 0.9834 | 0.9865 | 0.9923 | 0.8816 | 0.5574 | 0.2667 | 0.8515 | 0 | 0 | 0 |
| 93 | 5522.5400 | 0.5059 | 0.2590 | 0.8543 | 0.9841 | 0.9860 | 0.9924 | 0.8824 | 0.5576 | 0.2666 | 0.8514 | 0 | 0 | 0 |
| 94 | 5581.8900 | 0.5030 | 0.2569 | 0.8540 | 0.9834 | 0.9856 | 0.9923 | 0.8832 | 0.5578 | 0.2665 | 0.8514 | 0 | 0 | 0 |
| 95 | 5641.2800 | 0.5026 | 0.2564 | 0.8499 | 0.9841 | 0.9856 | 0.9924 | 0.8834 | 0.5577 | 0.2667 | 0.8514 | 0 | 0 | 0 |
| 96 | 5700.7200 | 0.5006 | 0.2556 | 0.8524 | 0.9842 | 0.9856 | 0.9924 | 0.8834 | 0.5573 | 0.2670 | 0.8514 | 0 | 0 | 0 |
| 97 | 5760.1500 | 0.5006 | 0.2566 | 0.8523 | 0.9834 | 0.9859 | 0.9924 | 0.8834 | 0.5574 | 0.2671 | 0.8514 | 0 | 0 | 0 |
| 98 | 5819.4400 | 0.4976 | 0.2574 | 0.8541 | 0.9835 | 0.9856 | 0.9924 | 0.8838 | 0.5573 | 0.2669 | 0.8514 | 0 | 0 | 0 |
| 99 | 5878.7200 | 0.4994 | 0.2562 | 0.8524 | 0.9849 | 0.9870 | 0.9931 | 0.8844 | 0.5570 | 0.2672 | 0.8514 | 0 | 0 | 0 |
| 100 | 5938.0700 | 0.5004 | 0.2568 | 0.8516 | 0.9849 | 0.9869 | 0.9930 | 0.8846 | 0.5567 | 0.2674 | 0.8513 | 0 | 0 | 0 |


### RT-DETR Training Metrics

Summary of training progress:

| metric | value |
| --- | --- |
| epochs | 29 |
| best_epoch_map | 27 |
| best_map | 0.8672 |
| best_epoch_map50 | 26 |
| best_map50 | 0.9892 |
| lowest_val_loss_epoch | 24 |
| lowest_val_loss | 3.2380 |
| final_epoch | 29 |
| final_train_loss | 6.1744 |
| final_val_loss | 3.3261 |
| final_map | 0.8637 |
| final_map50 | 0.9788 |

All epochs:

| Epoch | Training Loss | Validation Loss | Map | Map 50 |
| --- | --- | --- | --- | --- |
| 1 |  | 4.6236 | 0.7812 | 0.9742 |
| 2 | 8.7020 | 4.4012 | 0.8009 | 0.9806 |
| 3 | 8.5872 | 4.2336 | 0.8021 | 0.9714 |
| 4 | 8.5872 | 4.1174 | 0.8228 | 0.9874 |
| 5 | 8.2268 | 4.1232 | 0.8145 | 0.9737 |
| 6 | 8.0922 | 3.9591 | 0.8263 | 0.9885 |
| 7 | 8.0922 | 4.0253 | 0.8184 | 0.9790 |
| 8 | 7.7683 | 3.8048 | 0.8353 | 0.9883 |
| 9 | 7.6231 | 3.6923 | 0.8356 | 0.9790 |
| 10 | 7.6231 | 3.7186 | 0.8478 | 0.9883 |
| 11 | 7.3832 | 3.6487 | 0.8472 | 0.9890 |
| 12 | 7.2719 | 3.5769 | 0.8443 | 0.9888 |
| 13 | 7.2719 | 3.6054 | 0.8502 | 0.9884 |
| 14 | 7.1348 | 3.4694 | 0.8546 | 0.9890 |
| 15 | 7.0489 | 3.5463 | 0.8469 | 0.9787 |
| 16 | 7.0489 | 3.5186 | 0.8501 | 0.9794 |
| 17 | 6.9401 | 3.4120 | 0.8577 | 0.9889 |
| 18 | 6.7861 | 3.4426 | 0.8566 | 0.9885 |
| 19 | 6.7861 | 3.4392 | 0.8579 | 0.9880 |
| 20 | 6.6869 | 3.3871 | 0.8605 | 0.9881 |
| 21 | 6.5920 | 3.3782 | 0.8597 | 0.9888 |
| 22 | 6.5920 | 3.3712 | 0.8625 | 0.9886 |
| 23 | 6.5098 | 3.3195 | 0.8646 | 0.9889 |
| 24 | 6.4586 | 3.2380 | 0.8667 | 0.9882 |
| 25 | 6.4586 | 3.2969 | 0.8671 | 0.9879 |
| 26 | 6.3400 | 3.3212 | 0.8649 | 0.9892 |
| 27 | 6.2460 | 3.3058 | 0.8672 | 0.9879 |
| 28 | 6.2460 | 3.2714 | 0.8670 | 0.9856 |
| 29 | 6.1744 | 3.3261 | 0.8637 | 0.9788 |
